In [ ]:
train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

train_labels = train_set["SalePrice"]
train_set = train_set.drop("SalePrice", axis=1)


regressor = VotingRegressor(
    [
        ("xgb", XGBRegressor()),
        ("gbc", GradientBoostingRegressor()),
        ("lgbm", LGBMRegressor()),
        ("lasso", Lasso(alpha=0.001)),
        ("svr", SVR()),
        ("ridge", Ridge(alpha=10)),
        ("cat", CatBoostRegressor(silent=True, allow_writing_files=False)),
    ],
    n_jobs=-1,
)

full_pipeline = make_pipeline(
    preprocessing,
    SelectFromModel(ExtraTreesRegressor(random_state=42, bootstrap=True), threshold=1e-4),
    TransformedTargetRegressor(
        regressor,
        func=np.log,
        inverse_func=np.exp,
    ),
)

rmses = -cross_val_score(
    full_pipeline,
    train_set,
    train_labels,
    cv=3,
    scoring="neg_root_mean_squared_error",
    error_score="raise",  # type: ignore
)
pd.Series(rmses).describe()

# Jak sie dostac do elementow
# forest = best_model["transformedtargetregressor"].regressor_.named_estimators_.pop("forest")
# best_model["transformedtargetregressor"].regressor_.estimators_.remove(forest)
# vot_search.best_estimator_["selectfrommodel"].estimator_.n_features_in_

In [1]:
import joblib
final_model = joblib.load("models/best_stacking_regressor.pkl")
final_model

AttributeError: Can't get attribute 'CustomImputer' on <module '__main__'>